# 참고 DPO 트레이너 오류

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-53-c4c434e2b90d> in <cell line: 1>()
----> 1 dpo_trainer = DPOTrainer(
      2         model,
      3         model_ref,
      4         args=training_args,
      5         beta=0.1,

6 frames
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py in tokenize_row(self, feature, model)
    651             num_diff_len = abs(chosen_prompt_len_input_ids - rejected_prompt_len_input_ids)
    652             if num_diff_tokens > 1 or num_diff_len > 1:
--> 653                 raise ValueError(
    654                     "Chosen and rejected prompt_input_ids might only differ on the "
    655                     "last token due to tokenizer merge ops."

ValueError: Chosen and rejected prompt_input_ids might only differ on the last token due to tokenizer merge ops.



In [ ]:
# train_dat['chosen'][7512]

In [ ]:
# import re

# p_text = train_dat['prompt'][7512]

# c_text = train_dat['chosen'][7512]

# r_text = train_dat['rejected'][7512]

In [ ]:
#  def build_tokenized_answer(prompt, answer):
#         """
#         Llama tokenizer does satisfy `enc(a + b) = enc(a) + enc(b)`.
#         It does ensure `enc(a + b) = enc(a) + enc(a + b)[len(enc(a)):]`.
#         Reference:
#             https://github.com/EleutherAI/lm-evaluation-harness/pull/531#issuecomment-1595586257
#         """

#         full_tokenized = tokenizer(prompt + answer, add_special_tokens=False)
#         prompt_input_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]

#         answer_input_ids = full_tokenized["input_ids"][len(prompt_input_ids) :]
#         answer_attention_mask = full_tokenized["attention_mask"][len(prompt_input_ids) :]

#         # Concat tokens to form `enc(a) + enc(a + b)[len(enc(a)):]`
#         full_concat_input_ids = np.concatenate([prompt_input_ids, answer_input_ids])

#         # Prepare input tokens for token by token comparison
#         full_input_ids = np.array(full_tokenized["input_ids"])

#         if len(full_input_ids) != len(full_concat_input_ids):
#             raise ValueError("Prompt input ids and answer input ids should have the same length.")

#         # On some tokenizers, like Llama-2 tokenizer, there are occasions where tokens
#         # can be merged together when tokenizing prompt+answer. This could result
#         # on the last token from the prompt being different when tokenized on its own
#         # vs when done as prompt+answer.
#         response_token_ids_start_idx = len(prompt_input_ids)

#         # If tokenized prompt is different than both prompt+answer, then it means the
#         # last token has changed due to merging.
#         if prompt_input_ids != full_tokenized["input_ids"][:response_token_ids_start_idx]:
#             response_token_ids_start_idx -= 1

#         prompt_input_ids = full_tokenized["input_ids"][:response_token_ids_start_idx]
#         prompt_attention_mask = full_tokenized["attention_mask"][:response_token_ids_start_idx]

#         if len(prompt_input_ids) != len(prompt_attention_mask):
#             raise ValueError("Prompt input ids and attention mask should have the same length.")

#         answer_input_ids = full_tokenized["input_ids"][response_token_ids_start_idx:]
#         answer_attention_mask = full_tokenized["attention_mask"][response_token_ids_start_idx:]

#         return dict(
#             prompt_input_ids=prompt_input_ids,
#             prompt_attention_mask=prompt_attention_mask,
#             input_ids=answer_input_ids,
#             attention_mask=answer_attention_mask,
#         )


In [ ]:

# prompt_tokens = tokenizer(p_text, add_special_tokens=False)
# prompt_tokens = {f"prompt_{k}": v for k, v in prompt_tokens.items()}

# chosen_tokens = build_tokenized_answer(p_text, c_text)
# rejected_tokens = build_tokenized_answer(p_text, r_text)

# prompt_len_input_ids = len(prompt_tokens["prompt_input_ids"])

# chosen_prompt_len_input_ids = len(chosen_tokens["prompt_input_ids"])
# rejected_prompt_len_input_ids = len(rejected_tokens["prompt_input_ids"])
# prompt_len_input_ids = min(chosen_prompt_len_input_ids, rejected_prompt_len_input_ids)

# for k, v in prompt_tokens.items():
#     prompt_tokens[k] = v[:prompt_len_input_ids]

# # Make sure prompts only have one different token at most an
# # and length only differs by 1 at most
# num_diff_tokens = sum(
#     [a != b for a, b in zip(chosen_tokens["prompt_input_ids"], rejected_tokens["prompt_input_ids"])]
# )
# num_diff_len = abs(chosen_prompt_len_input_ids - rejected_prompt_len_input_ids)
# if num_diff_tokens > 1 or num_diff_len > 1:
#     raise ValueError(
#         "Chosen and rejected prompt_input_ids might only differ on the "
#         "last token due to tokenizer merge ops."
#     )

In [ ]:
# chosen_prompt_len_input_ids

In [ ]:
# rejected_prompt_len_input_ids

In [ ]:
# prompt_len_input_ids

# 참고 kaggle log loss

In [ ]:
import numpy as np
import pandas as pd
import pandas.api.types

import kaggle_metric_utilities

import sklearn.metrics

from typing import Sequence, Union, Optional


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, eps: Union[float, str]='auto', weights_column_name: Optional[str]=None, labels: Optional[Sequence]=None) -> float:
    # Skip sorting and equality checks for the row_id_column since that should already be handled
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    sample_weight = None
    if weights_column_name:
        if weights_column_name not in solution.columns:
            raise ValueError(f'The solution weights column {weights_column_name} is not found')
        sample_weight = solution.pop(weights_column_name).values
        if not pandas.api.types.is_numeric_dtype(sample_weight):
            raise ParticipantVisibleError('The solution weights are not numeric')

    if not pandas.api.types.is_numeric_dtype(submission.values):
        bad_dtypes = {x: submission[x].dtype  for x in submission.columns if not pandas.api.types.is_numeric_dtype(submission[x])}
        raise ParticipantVisibleError(f'Invalid submission data types found: {bad_dtypes}')

    if submission.max().max() > 1 or submission.min().min() < 0:
        raise ParticipantVisibleError('Submitted values were not valid probabilities')

    solution = solution.values
    submission = submission.values

    score_result = kaggle_metric_utilities.safe_call_score(sklearn.metrics.log_loss, solution, submission, eps=eps, sample_weight=sample_weight, labels=labels)

    return score_result

ModuleNotFoundError: No module named 'kaggle_metric_utilities'

# Install Library

In [ ]:
! pip install torch==2.0.1 peft accelerate trl optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.1
    Uninstalling transformers-4.42.1:
      Successfully uninstalled transformers-4.42.1


In [ ]:
! pip install bitsandbytes>=0.39.0

In [ ]:
! pip install --upgrade transformers

  Using cached transformers-4.42.1-py3-none-any.whl (9.3 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.20.0 requires transformers[sentencepiece]<4.42.0,>=4.26.0, but you have transformers 4.42.1 which is incompatible.


In [ ]:
! pip install --upgrade accelerate

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

# Load kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/dataset/lmsys/"

/content/drive/MyDrive/dataset/lmsys


In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


# EDA

In [ ]:
import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, GPTQConfig, DataCollatorWithPadding, BitsAndBytesConfig
from trl import DPOTrainer

In [ ]:
from transformers import AutoTokenizer

modelname = "mistralai/Mistral-7B-Instruct-v0.3"


tokenizer = AutoTokenizer.from_pretrained(modelname)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

In [ ]:


# test_df = train.copy()

# def tokenize_prompt(prompt, chosen, rejected):
#     prompt_tokens = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length = 4096)
#     chosen_tokens = tokenizer(chosen, return_tensors="pt", truncation=True, padding=True, max_length = 4096)
#     rejected_tokens = tokenizer(rejected, return_tensors="pt", truncation=True, padding=True, max_length = 4096)

#     return prompt_tokens['input_ids'], chosen_tokens['input_ids'], rejected_tokens['input_ids']

# p_list = []
# c_list = []
# r_list = []

# for i in range(len(test_df)):
#   prompt_tokens, chosen_tokens, rejected_tokens = tokenize_prompt(test_df["prompt"].iloc[i], test_df["response_a"].iloc[i], test_df["response_b"].iloc[i])
#   p_list.append(len(prompt_tokens[0]))
#   c_list.append(len(chosen_tokens[0]))
#   r_list.append(len(rejected_tokens[0]))

# test_df['p_len'] = p_list
# test_df['c_len'] = c_list
# test_df['r_len'] = r_list

In [ ]:
# test_df.head()

In [ ]:
# print(np.argmax(test_df['p_len']),"\n",np.argmax(test_df['c_len']),"\n",np.argmax(test_df['r_len']))
# print(np.mean(test_df['p_len']),"\n",np.mean(test_df['c_len']),"\n",np.mean(test_df['r_len']))


# Preprocessing for DPO

In [ ]:
train = train[train['winner_tie'] == 0]
train = train.reset_index(drop = True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39716 entries, 0 to 39715
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              39716 non-null  int64 
 1   model_a         39716 non-null  object
 2   model_b         39716 non-null  object
 3   prompt          39716 non-null  object
 4   response_a      39716 non-null  object
 5   response_b      39716 non-null  object
 6   winner_model_a  39716 non-null  int64 
 7   winner_model_b  39716 non-null  int64 
 8   winner_tie      39716 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 2.7+ MB


In [ ]:
chosen = []
rejected = []


for i in range(len(train)):

  if train['winner_model_a'].iloc[i] == 1:
    chosen.append(train['response_a'].iloc[i])
    rejected.append(train['response_b'].iloc[i])
  else :
    chosen.append(train['response_b'].iloc[i])
    rejected.append(train['response_a'].iloc[i])


train['chosen'] = chosen
train['rejected'] = rejected

train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,chosen,rejected
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,"[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o..."
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,"[""A marriage license and a marriage certificat...","[""A marriage license is a legal document that ..."
2,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,"[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c..."
3,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,"[""The best way to travel from Tel-Aviv to Jeru...","[""The best way to travel from Tel Aviv to Jeru..."
4,292873,vicuna-13b,gpt-4-0314,"[""Construct a rap battle, in the style of Epic...","[""[Zeus]\nYo, it's the king of the gods on the...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...",0,1,0,"[""(Verse 1 - Zeus)\n\nI'm the king of the gods...","[""[Zeus]\nYo, it's the king of the gods on the..."


In [ ]:
new_train = train[['prompt','chosen','rejected']]
new_train

,prompt,chosen,rejected
0,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o..."
1,"[""What is the difference between marriage lice...","[""A marriage license and a marriage certificat...","[""A marriage license is a legal document that ..."
2,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c..."
3,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel-Aviv to Jeru...","[""The best way to travel from Tel Aviv to Jeru..."
4,"[""Construct a rap battle, in the style of Epic...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...","[""[Zeus]\nYo, it's the king of the gods on the..."
...,...,...,...
39711,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig..."
39712,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba..."
39713,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us..."
39714,"[""If a bait contains 0,0025% bromadiolon then ...","[""As an AI language model, I do not promote or...","[""Bromadiolone is a rodenticide which is most ..."


In [ ]:
import json
new_train["prompt"] = new_train["prompt"].apply(lambda x: json.loads(x)[0])
new_train["chosen"] = new_train["chosen"].apply(lambda x: json.loads(x)[0])
new_train["rejected"] = new_train["rejected"].apply(lambda x: json.loads(x)[0])

<ipython-input-13-d8139dda7f0d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train["prompt"] = new_train["prompt"].apply(lambda x: json.loads(x)[0])
<ipython-input-13-d8139dda7f0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train["chosen"] = new_train["chosen"].apply(lambda x: json.loads(x)[0])
<ipython-input-13-d8139dda7f0d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
new_train = new_train.dropna()

In [ ]:
from sklearn.model_selection import train_test_split


train_dat, val_dat = train_test_split(new_train, test_size=0.2, random_state=42)

train_dat = train_dat.reset_index(drop=True)
val_dat = val_dat.reset_index(drop=True)

In [ ]:
train_dat = train_dat.drop(7512, axis = 0)
train_dat = train_dat.reset_index(drop=True)

In [ ]:
val_dat = val_dat.drop(1428, axis = 0)
val_dat = val_dat.reset_index(drop=True)

In [ ]:
train_dat = Dataset.from_pandas(train_dat)
train_dat

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 31753
})

In [ ]:
val_dat = Dataset.from_pandas(val_dat)
val_dat

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 7938
})

# Training DPO Model



In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# quantization_config = GPTQConfig(
#     bits=4,
#     dataset="wikitext2",
#     disable_exllama=True
# )

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    modelname,
    quantization_config=quantization_config
)

# model_ref = AutoModelForCausalLM.from_pretrained(
#     modelname,
#     quantization_config=quantization_config
# )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# model.push_to_hub("dongseon/kaggle_mistral-quant")

In [ ]:
# model_name = "dongseon/kaggle_mistral-quant"

# quantization_config = GPTQConfig(
#     bits=4,
#     dataset="wikitext2",
#     disable_exllama=True
# )

# # 모델 로드
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     quantization_config=quantization_config
# )

# model_ref = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     quantization_config=quantization_config
# )

In [ ]:
peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["o_proj", "v_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
peft_config.inference_mode = False

In [ ]:
model = prepare_model_for_kbit_training(model)
model.config.use_cache=False
model.gradient_checkpointing_enable()
model.config.pretraining_tp=1
model = get_peft_model(model, peft_config)

In [ ]:
# model_ref = prepare_model_for_kbit_training(model_ref)
# model_ref.config.use_cache=False
# model_ref.gradient_checkpointing_enable()
# model_ref.config.pretraining_tp=1
# model_ref = get_peft_model(model_ref, peft_config)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1024, bias=False)
                )
                (lora_embeddi

In [ ]:
from trl import DPOConfig

training_args = DPOConfig(
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    # max_steps=50,
    remove_unused_columns=False,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    eval_strategy="steps",
    logging_first_step=True,
    logging_steps=10,
    output_dir="dongseon/kaggle_mistral-dpo",
    optim="paged_adamw_32bit",
    warmup_steps=2,
    fp16=True,
    # push_to_hub=True
)

In [ ]:
dpo_trainer = DPOTrainer(
        model,
        # model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_dat,
        eval_dataset=val_dat,
        tokenizer=tokenizer,
        max_length=384,
        max_target_length=384,
        max_prompt_length=384,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_length, max_target_length, max_prompt_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:358: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:371: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:384: UserWarning: You passed `max_target_length` to the DPOTrainer, the value you passed will override the one in the `DPOC

Map:   0%|          | 0/31753 [00:00<?, ? examples/s]

Map:   0%|          | 0/7938 [00:00<?, ? examples/s]

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

81

In [ ]:
dpo_trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.692600,0.691224,0.005151,0.001156,0.312521,0.003994,-80.685326,-92.052170,-2.493677,-2.521866
20,0.691000,0.689345,0.001900,-0.006161,0.319025,0.008061,-80.758499,-92.084679,-2.490178,-2.518502
30,0.687000,0.686745,-0.006826,-0.020874,0.324564,0.014048,-80.905624,-92.171944,-2.485475,-2.513913
40,0.689600,0.684452,-0.015749,-0.035650,0.325445,0.019900,-81.053391,-92.261177,-2.481327,-2.509869
50,0.688800,0.682669,-0.026923,-0.051818,0.326955,0.024895,-81.215065,-92.372902,-2.477535,-2.506176
60,0.680700,0.680100,-0.045573,-0.077744,0.327711,0.032171,-81.474327,-92.559410,-2.476197,-2.504914
70,0.681000,0.677108,-0.083879,-0.126440,0.325990,0.042561,-81.961288,-92.942467,-2.473298,-2.502063


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.692600,0.691224,0.005151,0.001156,0.312521,0.003994,-80.685326,-92.052170,-2.493677,-2.521866
20,0.691000,0.689345,0.001900,-0.006161,0.319025,0.008061,-80.758499,-92.084679,-2.490178,-2.518502
30,0.687000,0.686745,-0.006826,-0.020874,0.324564,0.014048,-80.905624,-92.171944,-2.485475,-2.513913
40,0.689600,0.684452,-0.015749,-0.035650,0.325445,0.019900,-81.053391,-92.261177,-2.481327,-2.509869
50,0.688800,0.682669,-0.026923,-0.051818,0.326955,0.024895,-81.215065,-92.372902,-2.477535,-2.506176
60,0.680700,0.680100,-0.045573,-0.077744,0.327711,0.032171,-81.474327,-92.559410,-2.476197,-2.504914
70,0.681000,0.677108,-0.083879,-0.126440,0.325990,0.042561,-81.961288,-92.942467,-2.473298,-2.502063


In [ ]:
dpo_trainer.push_to_hub("dongseon/kaggle_mistral-dpo")

In [ ]:

# GPU 메모리 비우기
torch.cuda.empty_cache()


# Preprocessing for Classifier

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [ ]:
clf_train = train[['prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie']]

In [ ]:
import json
clf_train["prompt"] = clf_train["prompt"].apply(lambda x: json.loads(x)[0])
clf_train["response_a"] = clf_train["response_a"].apply(lambda x: json.loads(x)[0])
clf_train["response_b"] = clf_train["response_b"].apply(lambda x: json.loads(x)[0])

<ipython-input-12-6d6c27551bad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_train["prompt"] = clf_train["prompt"].apply(lambda x: json.loads(x)[0])
<ipython-input-12-6d6c27551bad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_train["response_a"] = clf_train["response_a"].apply(lambda x: json.loads(x)[0])
<ipython-input-12-6d6c27551bad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [ ]:
clf_train = clf_train.dropna()
clf_train = clf_train.reset_index(drop = True)

In [ ]:
clf_train['new_text'] = [ "### prompt: "+clf_train['prompt'][x]+" ### response_a: "+clf_train['response_a'][x]+" ### response_b: "+clf_train['response_b'][x] for x in range(len(clf_train)) ]

In [ ]:
clf_train['target'] = [[clf_train['winner_model_a'][x],clf_train['winner_model_b'][x],clf_train['winner_tie'][x]] for x in range(len(clf_train)) ]

In [ ]:
clf_train = clf_train[['new_text','target']]

In [ ]:
clf_train.head()

,new_text,target
0,### prompt: Is it morally right to try to have...,"[1, 0, 0]"
1,### prompt: What is the difference between mar...,"[0, 1, 0]"
2,### prompt: explain function calling. how woul...,"[0, 0, 1]"
3,### prompt: How can I create a test set for a ...,"[1, 0, 0]"
4,### prompt: What is the best way to travel fro...,"[0, 1, 0]"


In [ ]:

def cl(x):
  if x == [1,0,0]:
    return 0
  elif x == [0,1,0]:
    return 1
  else :
    return 2

clf_train['labels'] = clf_train['target'].apply(lambda x : cl(x))

In [ ]:
from sklearn.model_selection import train_test_split


t_dat, v_dat = train_test_split(clf_train, test_size=0.2, random_state=42, stratify = clf_train['labels'])

t_dat = t_dat.reset_index(drop=True)
v_dat = v_dat.reset_index(drop=True)

In [ ]:
t_dat = t_dat.drop( labels= 'target' , axis = 1)
v_dat = v_dat.drop( labels= 'target' , axis = 1)

In [ ]:
# t_dat = t_dat.iloc[:1000,:]
# v_dat = v_dat.iloc[:100,:]

In [ ]:
t_dat.head()

,new_text,labels
0,### prompt: Can you tell another Richard Pryor...,2
1,"### prompt: paraphrase: ""Write a social media ...",1
2,### prompt: Hi there please help me write a ce...,1
3,### prompt: funcion generatiu de moments d'una...,0
4,### prompt: I'm interested in the sum from n= ...,2


In [ ]:
np.unique(t_dat['labels'])


array([0, 1, 2])

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch


model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['new_text']
        self.targets = self.df['labels']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.LongTensor([self.targets[index]])
        }

In [ ]:
max_len = 2048

In [ ]:
# trainer api

train_dataset = CustomDataset(t_dat, tokenizer, max_len)
val_dataset = CustomDataset(v_dat, tokenizer, max_len)

In [ ]:
train_dataset.__getitem__(0)

{'input_ids': tensor([    1,   774, 11510,  ...,     2,     2,     2]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor([2])}

In [ ]:
tokenizer.decode([1])

'<s>'

In [ ]:
# low train

train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

valid_data_loader = torch.utils.data.DataLoader(val_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

# Training Classifier Model

In [ ]:
from torch import nn
import torch.nn.functional as F
from transformers import BitsAndBytesConfig
import torch.nn.init as init


model_name = "dongseon/kaggle_mistral-dpo"

loss_fn = nn.CrossEntropyLoss()

class PeftModelForClassification(nn.Module):
    def __init__(self, peft_model, config):  # config 추가
        super(PeftModelForClassification, self).__init__()
        self.config = config  # config 설정
        self.peft_model = peft_model
        self.dropout = torch.nn.Dropout(0.2)
        self.classifier = nn.Linear(config.hidden_size, 3)  # config.hidden_size를 사용하여 linear layer 크기 설정
        self.classifier.weight.data = self.classifier.weight.data.to(torch.float16)
        self.classifier.bias.data = self.classifier.bias.data.to(torch.float16)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.peft_model(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states[-1]
        pooled_output = hidden_states.mean(dim=1)
        output_dropout = self.dropout(pooled_output)
        logits = self.classifier(output_dropout)
        loss = None
        if labels is not None:
          labels = labels.squeeze()
          loss = loss_fn(logits, labels)
        return loss, logits

peft_model = AutoPeftModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
    output_hidden_states=True,

)

# PEFT 모델에 분류기 추가할 때 config도 전달
model = PeftModelForClassification(peft_model, peft_model.config).to("cuda")

adapter_config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
# 분류기를 제외한 모든 가중치 프리징
for param in model.peft_model.parameters():
    param.requires_grad = False
# for param in model.classifier.parameters():
#     param.requires_grad = True

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad == True)
print(f"Total trainable parameters: {total_params}")
# Total trainable parameters: 12291

Total trainable parameters: 12291


In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, Shape: {param.shape}, Requires grad: {param.requires_grad}")


Parameter name: peft_model.base_model.model.model.embed_tokens.weight, Shape: torch.Size([32000, 4096]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight, Shape: torch.Size([8, 4096]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight, Shape: torch.Size([4096, 8]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight, Shape: torch.Size([8, 4096]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight, Shape: torch.Size([1024, 8]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.input_layernorm.weight, Shape: torch.Size([4096]), Requires grad: False
Parameter name: peft_model.base_model.model.model.layers.0.post_attention_layernorm.weight, Shape: torch.Size([4096]), Requires grad: False
Par

1. 선형 분류기하나만 추가 및 분류기 외 그래디언트 false 후 학습 시모델이 너무 단조로워, 학습 로스가 0으로 학습이 안됨  
2. 시퀀셜 레이어를 추가하거나 헤드(채널)를 추가 하는 등의 작업이 필요해보여 프리징은 제외  

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc=accuracy_score(labels , preds)
  return {"accuracy":acc, "f1": f1}

In [ ]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, max_length = max_len,
                                                   padding='max_length',return_tensors='pt')

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# 훈련 인수 설정
training_args = TrainingArguments(
    output_dir="dongseon/kaggle_mistral-clf",
    max_step = 30,
    # num_train_epochs=1,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    warmup_ratio=0.1,
    weight_decay=0.1,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    optim="adamw_hf",
    learning_rate=2e-5,
    push_to_hub=True,
    save_safetensors= False,
    load_best_model_at_end=True
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    # data_collator = data_collator
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub(
    "dongseon/kaggle_mistral-clf"
)

In [ ]:

# GPU 메모리 비우기
torch.cuda.empty_cache()


## low training

In [ ]:
# device = "cuda"
# LEARNING_RATE = 1e-5
# best_model_path = "/content/best_model.pt"
# checkpoint_path = "/content"
# EPOCHS = 5

In [ ]:
# import torch.optim as optim
# from torch.cuda.amp import GradScaler, autocast
# # Optimizer 설정
# optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# # AMP를 위한 GradScaler 설정
# scaler = GradScaler()

In [ ]:
# def load_ckp(checkpoint_fpath, model, optimizer):
#     checkpoint = torch.load(checkpoint_fpath)
#     model.load_state_dict(checkpoint['state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer'])
#     valid_loss_min = checkpoint['valid_loss_min']
#     return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

# def save_ckp(state, is_best, checkpoint_path, best_model_path):
#     f_path = checkpoint_path
#     torch.save(state, f_path)
#     if is_best:
#         best_fpath = best_model_path
#         shutil.copyfile(f_path, best_fpath)

In [ ]:
# def train_model(n_epochs, training_loader, validation_loader,model,
#                 optimizer,checkpoint_path,  best_model_path):

#   for epoch in range(1, n_epochs+1):
#     train_loss = 0
#     valid_loss = 0

#     model.train()
#     print('############# Epoch {}: Training Start   #############'.format(epoch))
#     for batch_idx, data in enumerate(training_loader):

#         input_ids = data['input_ids'].to(device)
#         attention_mask = data['attention_mask'].to(device)
#         labels = data['labels'].to(device)

#         loss, _ = model(input_ids, attention_mask, labels)
#         optimizer.zero_grad()
#         # Loss 역전파
#         loss.backward()
#         optimizer.step()

#         train_loss += loss.item()

#     print('############# Epoch {}: Training End     #############'.format(epoch))


#     print('############# Epoch {}: Validation Start   #############'.format(epoch))
#     model.eval()

#     with torch.no_grad():
#       for batch_idx, data in enumerate(validation_loader):

#         input_ids = data['input_ids'].to(device)
#         attention_mask = data['attention_mask'].to(device)
#         labels = data['labels'].to(device)

#         loss, _ = model(input_ids, attention_mask, labels)

#         valid_loss += loss.item()
#     print('############# Epoch {}: Validation End     #############'.format(epoch))

#     train_loss = train_loss / len(training_loader)
#     valid_loss = valid_loss / len(validation_loader)

#     print('Epoch: {} \tAverage Training Loss: {:.6f}'.format(epoch, train_loss))
#     print('Epoch: {} \tAverage Validation Loss: {:.6f}'.format(epoch, valid_loss))


#     checkpoint = {
#           'epoch': epoch + 1,
#           'valid_loss_min': valid_loss,
#           'state_dict': model.state_dict(),
#           'optimizer': optimizer.state_dict()
#     }

#       # save checkpoint
#     save_ckp(checkpoint, True, checkpoint_path, best_model_path)
#     new_state_dict = {key: value for key, value in model.state_dict().items()}    # if key != 'linear.weight' and key != 'linear.bias'}
#     torch.save(new_state_dict, '/content/drive/MyDrive/best_model_weights.pt')
#     ## TODO: save the model if validation loss has decreased
#     if valid_loss <= valid_loss_min:
#       print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
#       # save checkpoint as best model
#       save_ckp(checkpoint, True, checkpoint_path, best_model_path)
#       torch.save(new_state_dict, '/content/best_model_weights.pt')
#       valid_loss_min = valid_loss

#     print('############# Epoch {}  Done   #############\n'.format(epoch))

#   return model

In [ ]:
# trained_model = train_model(EPOCHS, train_data_loader, valid_data_loader model, optimizer,checkpoint_path, best_model_path)

# Inference

In [ ]:
inputs = next(iter(train_data_loader))

In [ ]:
inputs['input_ids'].shape

torch.Size([64, 512])

In [ ]:
outputs = peft_model(inputs['input_ids'].to("cuda"), attention_mask=inputs['attention_mask'].to("cuda"))

In [ ]:
outputs.hidden_states[-1].shape

torch.Size([64, 512, 4096])

In [ ]:
outputs.hidden_states[-1].mean(dim=1).shape

torch.Size([64, 4096])

In [ ]:
from torch import nn
classifier = nn.Linear(4096, 3 , device = "cuda")
classifier.weight.data = classifier.weight.data.to(torch.float16)
classifier.bias.data = classifier.bias.data.to(torch.float16)

hidden_states = outputs.hidden_states[-1]
pooled_output = hidden_states.mean(dim=1)  # 평균 풀링

# pooled_output = pooled_output.to(torch.float16)

In [ ]:
pooled_output.dtype

torch.float16

In [ ]:
logits = classifier(pooled_output)
logits.shape

torch.Size([64, 3])

In [ ]:
labels = inputs['labels'].to("cuda")
labels.squeeze().shape

torch.Size([64])

In [ ]:
labels[0]

tensor([0], device='cuda:0')

In [ ]:
loss = loss_fn(logits, labels.squeeze())

In [ ]:
loss

tensor(1.8740, device='cuda:0', dtype=torch.float16,
       grad_fn=<NllLossBackward0>)

In [ ]:
loss.backward()

In [ ]:
_, logit = model(inputs['input_ids'].to("cuda"), attention_mask=inputs['attention_mask'].to("cuda"))

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB (GPU 0; 39.56 GiB total capacity; 34.21 GiB already allocated; 348.81 MiB free; 38.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
logit.shape

# Test Inference

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoConfig
from torch import nn


model_name = "dongseon/kaggle_mistral-clf"

# PEFT 모델에 분류기 추가
# model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
!ls

dongseon  logs	sample_submission.csv  test.csv  train.csv


In [ ]:
# from huggingface_hub import hf_hub_download
# hf_hub_download(repo_id=model_name, filename="pytorch_model.bin")

'/root/.cache/huggingface/hub/models--dongseon--kaggle_mistral-clf/snapshots/89397e9505820aca04825aaae0aafa32401b46ee/pytorch_model.bin'

In [ ]:
model = PeftModelForClassification(peft_model, peft_model.config).to("cuda")

In [ ]:
model_dict = torch.load('./dongseon/kaggle_mistral-clf/pytorch_model.bin', map_location=torch.device('cuda'))

In [ ]:
model.load_state_dict(model_dict)

NameError: name 'model' is not defined

In [ ]:
model.eval()

PeftModelForClassification(
  (peft_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): MistralForCausalLM(
        (model): MistralModel(
          (embed_tokens): Embedding(32000, 4096)
          (layers): ModuleList(
            (0-31): 32 x MistralDecoderLayer(
              (self_attn): MistralAttention(
                (rotary_emb): MistralRotaryEmbedding()
                (k_proj): QuantLinear()
                (o_proj): QuantLinear()
                (q_proj): lora.QuantLinear(
                  (base_layer): QuantLinear()
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                  (lora_emb

In [ ]:
# model.load_state_dict(torch.load("./pytorch_model.bin"))

In [ ]:
test = pd.read_csv('test.csv')
len(test)

3

In [ ]:
import json
test["prompt"] = test["prompt"].apply(lambda x: json.loads(x)[0])
test["response_a"] = test["response_a"].apply(lambda x: json.loads(x)[0])
test["response_b"] = test["response_b"].apply(lambda x: json.loads(x)[0])

In [ ]:
test['new_text'] = [ "### prompt: "+test['prompt'][x]+" ### response_a: "+test['response_a'][x]+" ### response_b: "+test['response_b'][x] for x in range(len(test)) ]

In [ ]:
test.head()

,id,prompt,response_a,response_b,new_text
0,136060,"I have three oranges today, I ate an orange ye...",You have two oranges today.,You still have three oranges. Eating an orange...,"### prompt: I have three oranges today, I ate ..."
1,211333,You are a mediator in a heated political debat...,Thank you for sharing the details of the situa...,Mr Reddy and Ms Blue both have valid points in...,### prompt: You are a mediator in a heated pol...
2,1233961,How to initialize the classification head when...,When you want to initialize the classification...,To initialize the classification head when per...,### prompt: How to initialize the classificati...


In [ ]:
test_df = test.copy()

def tokenize_prompt(prompt, chosen, rejected):
    prompt_tokens = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length = 4096)
    chosen_tokens = tokenizer(chosen, return_tensors="pt", truncation=True, padding=True, max_length = 4096)
    rejected_tokens = tokenizer(rejected, return_tensors="pt", truncation=True, padding=True, max_length = 4096)

    return prompt_tokens['input_ids'], chosen_tokens['input_ids'], rejected_tokens['input_ids']

p_list = []
c_list = []
r_list = []

for i in range(len(test_df)):
  prompt_tokens, chosen_tokens, rejected_tokens = tokenize_prompt(test_df["prompt"].iloc[i], test_df["response_a"].iloc[i], test_df["response_b"].iloc[i])
  p_list.append(len(prompt_tokens[0]))
  c_list.append(len(chosen_tokens[0]))
  r_list.append(len(rejected_tokens[0]))

test_df['p_len'] = p_list
test_df['c_len'] = c_list
test_df['r_len'] = r_list



In [ ]:
print(np.max(test_df['p_len']),"\n",np.max(test_df['c_len']),"\n",np.max(test_df['r_len']))
print(np.mean(test_df['p_len']),"\n",np.mean(test_df['c_len']),"\n",np.mean(test_df['r_len']))

110 
 383 
 546
58.0 
 220.33333333333334 
 220.66666666666666


In [ ]:

class CustomDataset2(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['new_text']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

In [ ]:
test_dataset = CustomDataset2(test, tokenizer, max_len)



In [ ]:
test_dataset.__len__()

3

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0
)

In [ ]:
test_dataset[0]

{'input_ids': tensor([    1,   774, 11510,  ...,     2,     2,     2]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}

In [ ]:
aaa = next(iter(test_data_loader))
aaa

{'input_ids': tensor([[    1,   774, 11510,  ...,     2,     2,     2],
         [    1,   774, 11510,  ...,     2,     2,     2],
         [    1,   774, 11510,  ...,     2,     2,     2]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:

device = "cuda"

with torch.no_grad():
  input_ids = aaa['input_ids'].to(device, dtype = torch.long)
  attention_mask = aaa['attention_mask'].to(device, dtype = torch.long)
  _, logits = model(input_ids, attention_mask)


In [ ]:
logits

tensor([[ 1.2158, -0.6289, -2.8066],
        [ 1.3594, -4.3008, -5.8672],
        [-0.4314, -1.4043, -3.4805]], device='cuda:0', dtype=torch.float16)

In [ ]:
device = "cuda"

target_list = []

for data in test_data_loader:
  with torch.no_grad():
    input_ids = data['input_ids'].to(device, dtype = torch.long)
    attention_mask = data['attention_mask'].to(device, dtype = torch.long)
    _, logits = model(input_ids, attention_mask)
    softmax_logits = torch.nn.functional.softmax(logits, dim=1)
    target_list.append(softmax_logits)

In [ ]:
target_list[0].cpu().detach().numpy()[:,0]

array([0.9956, 0.7017, 0.8506], dtype=float16)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['winner_model_a'] = target_list[0].cpu().detach().numpy()[:,0]
sub['winner_model_b'] = target_list[0].cpu().detach().numpy()[:,1]
sub['winner_tie'] = target_list[0].cpu().detach().numpy()[:,2]

In [ ]:
sub.to_csv('submission.csv', index=False)
